One can start from this notebook, when working with Raman measurements. Just change the data path below to point to your exported Raman measurements in text files.
One needs to export from Witec project as *table* and make sure to have the Raman shift dimensions in cm$^{-1}$.

In [3]:
%load_ext autoreload
%autoreload 2

In [86]:
import ramantools as rt
import glob, sys, re
import pylab as pl
import numpy as np
import xarray as xr

In [5]:
%matplotlib widget

# Loading a Raman map

For this we need to load the map, exported as *table* from *Witec Project* or *Witec Control*, as well as the text file next to the map. This text file is used to read the metadata of the Raman measurement.

In [14]:
data_path = r'C:\Users\zrbyt\OneDrive - energia.mta.hu\Documents\work_MFA\ABC\G peak anomaly\Raman measurements'
data_path += '\\'
sys.path.append(data_path)

Need to add `r` in front of the file path to treat it as a *raw string*, meaning that `\` is treated as a character.

Using `glob` to read the contents to the data folder above. The use of `glob` helps with searching for files according to patterns. See below, where we identify the map called: *ABC_36* and load its contents, including the text file containing the metadata.

In [ ]:
# list all the files in the data folder
glob.glob(data_path + '*')

In [17]:
map_path = glob.glob(data_path + '*ABC_36*' + '*Data*')[0]
info_path = glob.glob(data_path + '*ABC_36*' + '*Information*')[0]

Load the Raman map by passing the path of the map file and the info file to the `ramanmap` class. Let's call it `m`.

In [73]:
m = rt.ramanmap(map_path, info_path)

This returns a `ramanmap` object, which contains as variables:
- the raw imported, Raman measurements are stored in: `m.map`
- all the metadata in: `m.metadata`. The `m.print_metadata()` function, prints the contents of the info file.
- The Raman shift of the measurement is stored in `m.ramanshift`

In addition to the above, the `ramanmap` object, called `m` in this example has the following vaiables, mostly containing the measurement metadata.

In [83]:
list(m.__dict__)

['metadata',
 'mapname',
 'pixel_x',
 'pixel_y',
 'size_x',
 'size_y',
 'date',
 'time',
 'samplename',
 'laser',
 'itime',
 'grating',
 'objname',
 'objmagn',
 'positioner_x',
 'positioner_y',
 'ramanshift',
 'map']

In [87]:
m.ramanshift

array([-639.473, -635.679, -631.886, ..., 4014.87 , 4017.1  , 4019.33 ])